In [ ]:
import sqlglot
from IPython.display import display, Markdown

In [ ]:
import sys
from IPython.display import display, Markdown

def display_ansi_formatted_text(ansi_text, html_font_size="30"):
    if 'ipykernel' in sys.modules:
        md_text = ansi_text.replace("\x1b[4m", "**").replace("\x1b[0m", "**")
        display(Markdown(md_text))
    else:
        # Directly print the ANSI text in console environments
        print(ansi_text)

# Example usage with a string from sqlglot
ansi_string = "SELECT baz FROM \x1b[4mt\x1b[0m"
display_ansi_formatted_text(ansi_string)


In [ ]:
def clean_sql_simple(sql):
    return sqlglot.transpile(sql, read="duckdb", write="duckdb")[0]

In [ ]:
def check_sql(sql):
    try:
        return sqlglot.transpile(sql, read="duckdb", write="duckdb")[0]
    except sqlglot.errors.ParseError as e:
        print(f"SQL ERROR: {e.errors[0]['description']}\nQuery: '{sql[:e.errors[0]['col']]}'")
        #print(e.errors)
        return None

In [ ]:
test_sql_with_error = "SELECT foo FROM (SELECT baz FROM t"

In [ ]:
check_sql(test_sql_with_error)

In [ ]:
formatting_sql = "selECT   * from Tablename"

In [ ]:
check_sql(formatting_sql)

In [ ]:
def validate_sql(sql):
    try:
        expressions = sqlglot.parse(sql, dialect="duckdb")
        print(f"Expressions: {expressions}")
        if len(expressions) > 1:
            raise ValueError("Only one statement is allowed")
        disallowed_keywords = [
            "INSERT",
            "UPDATE",
            "DELETE",
            "DROP",
            "EXEC",
            "CALL",
            "ALTER",
            "GRANT",
        ]
        for expression in expressions:
            if any(token in expression.sql().upper() for token in disallowed_keywords):
                raise ValueError("Disallowed SQL keywords detected")
        return True, "Query is valid"
    except Exception as e:
        return False, str(e)

In [ ]:
validate_sql(formatting_sql)

In [ ]:
validate_sql(test_sql_with_error)

In [ ]:
def check_and_validate_sql(sql):
    validate = validate_sql(sql)
    if validate[0]:
        return check_sql(sql)
    else:
        return display_ansi_formatted_text(f"#### ERROR: {validate[1]}")
        #return check_sql(sql)

In [ ]:
check_and_validate_sql(formatting_sql)

In [ ]:
check_and_validate_sql(test_sql_with_error)

In [ ]:
import tomllib
from pathlib import Path
from loguru import logger

def read_config(parent_dir=0):
    config_toml = Path.cwd().parents[parent_dir] / "app" / "app.toml"
    if not config_toml.exists():
        logger.error(f"File Not Found - {str(config_toml)}")
        raise FileNotFoundError(f"{str(config_toml)}")
    with open(config_toml, 'rb') as f:
        logger.info(f"Config loaded from {str(config_toml)}")
        return tomllib.load(f)


In [ ]:
config = read_config()

In [ ]:
config['app']

In [ ]:
config['sql']['disallowed_keywords']